

L’application utilisée par le banquier nous envoie les informations suivantes:

{ "RevolvingUtilizationOfUnsecuredLines": 0.01703559, "NumberOfDependents": 1, "DebtRatio": 0, "age": 42, "NumberOfOpenCreditLinesAndLoans": 6, "NumberRealEstateLoansOrLines": 1, "NumberOfTime30-59DaysPastDueNotWorse": 1, "NumberOfTime60-89DaysPastDueNotWorse": 0, "NumberOfTimes90DaysLate": 0 }

- Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
- Committez, et envoyez moi le lien pour que je retrouve facilement

- Create a new test set with new data from one input
- Do all modifications (same as the train_set)
- BigMl:
        - create sources
        - create datasets
        - train a model (ensemble) on the train_set
        - prediction on the new test_set
        - show the prediction



In [3]:
from pandas import read_csv
import pandas as pd

In [4]:
# Create dataframe with columns
df = pd.DataFrame(columns = ['Id',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'SeriousDlqin2yrs',
 'NumberCreditLines'])
df

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,NumberCreditLines


In [5]:
# Create dataframe with new input
input = {'Id': [1],
         'RevolvingUtilizationOfUnsecuredLines': [0.01703559], 
         'age': [42], 
         'NumberOfTime30-59DaysPastDueNotWorse': [1], 
         'DebtRatio': [0], 
         'NumberOfOpenCreditLinesAndLoans': [6], 
         'NumberOfTimes90DaysLate': [0], 
         'NumberRealEstateLoansOrLines': [1], 
         'NumberOfTime60-89DaysPastDueNotWorse': [0], 
         'NumberOfDependents': [1],
       }
df_input = pd.DataFrame.from_dict(input)
df_input

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.017036,42,1,0,6,0,1,0,1


In [6]:
#Concatenate to fill the table with new data
df_exo = pd.concat([df, df_input], sort = False)
df_exo

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,NumberCreditLines
0,1,0.017036,42,1,0,NaN,6,0,1,0,1,NaN,NaN


In [8]:
#Call the function for modifications and apply to exo dataframe
from ipynb.fs.full.Modifications_function import modification

df_change = modification(df_exo)
df_change

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,NumberCreditLines,NumberOfTotalTime,MonthlyDebt,MonthlyBalance,RatioIncomeDependents
0,1,0.017036,42,1,0,0,6,0,1,0,1,NaN,NaN,1,0,0,0.0


In [9]:
export_csv = df_change.to_csv (r'exo_testset.csv', header=True)

In [10]:
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

In [11]:
# BigML prediction
source = api.create_source('fulltrain_dataframe.csv')
api.ok(source)
dataset_train = api.create_dataset(source)
api.ok(dataset_train)
test_source = api.create_source('exo_testset.csv')
api.ok(test_source)
dataset_test = api.create_dataset(test_source)
api.ok(dataset_test)
model = api.create_ensemble(dataset_train, {"objective_field":"SeriousDlqin2yrs"})
api.ok(model)
batch_prediction = api.create_batch_prediction(model, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True})
api.ok(batch_prediction)
api.download_batch_prediction(batch_prediction, filename='exo_prediction.csv')
df_exo_final = pd.read_csv('exo_prediction.csv', index_col = 0)
df_exo_final

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,...,SeriousDlqin2yrs,NumberCreditLines,NumberOfTotalTime,MonthlyDebt,MonthlyBalance,RatioIncomeDependents,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
field1,,,,,,,,,,,,,,,,,,,,,
0,1,0.017036,42,1,0,0,6,0,1,0,...,NaN,NaN,1,0,0,0,0,0.95663,0.96607,0.03393


scikit-learn

In [16]:
from pandas import read_csv
from pprint import pprint 
train_prepared = read_csv('fulltrain_dataframe.csv', index_col=0)
val_prepared = read_csv('exo_testset.csv', index_col=0)

In [17]:
target_column = 'SeriousDlqin2yrs'
y_train = train_prepared[target_column].values
print(y_train)

X_train = train_prepared.drop(target_column, axis=1).values
pprint(X_train)

[1 0 0 ... 0 0 0]
array([[7.66126609e-01, 4.50000000e+01, 2.00000000e+00, ...,
        7.32319702e+03, 1.79680298e+03, 4.56000000e+03],
       [9.57151019e-01, 4.00000000e+01, 0.00000000e+00, ...,
        3.16878123e+02, 2.28312188e+03, 2.60000000e+03],
       [6.58180140e-01, 3.80000000e+01, 1.00000000e+00, ...,
        2.58914887e+02, 2.78308511e+03,            inf],
       ...,
       [2.46043918e-01, 5.80000000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00,            nan],
       [0.00000000e+00, 3.00000000e+01, 0.00000000e+00, ...,
        0.00000000e+00, 5.71600000e+03,            inf],
       [8.50282951e-01, 6.40000000e+01, 0.00000000e+00, ...,
        2.03875009e+03, 6.11924991e+03,            inf]])


In [18]:
y_val = val_prepared[target_column].values
X_val = val_prepared.drop(target_column, axis=1).values
print(y_val)
print(X_val)

[nan]
[[1.703559e-02 4.200000e+01 1.000000e+00 0.000000e+00 0.000000e+00
  6.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 1.000000e+00
  1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]]


In [19]:
import pickle
from xgboost import XGBClassifier

filename = 'sklearn_model.sav'
loaded_model = pickle.load(open(filename, 'rb')) # Load the model already trained
result = loaded_model.predict(X_val)
print(result)

y_val_proba = loaded_model.predict_proba(X_val)
print(y_val_proba)
y_val_proba = y_val_proba[:,1]
print(y_val_proba)

[0]
[[0.95149064 0.04850933]]
[0.04850933]
